In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

2024-06-03 07:51:16.512671: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-03 07:51:16.543370: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 07:51:16.963515: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('MAC000002.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10153 entries, 0 to 10152
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   time_idx                     10153 non-null  int64  
 1   Energy_kwh                   10153 non-null  float64
 2   house_hold                   10153 non-null  object 
 3   temperature                  10151 non-null  float64
 4   apparentTemperature          10151 non-null  float64
 5   bool_weather_missing_values  10153 non-null  int64  
 6   year                         10153 non-null  int64  
 7   month                        10153 non-null  int64  
 8   day                          10153 non-null  int64  
 9   hour                         10153 non-null  int64  
 10  dayofweek_num                10153 non-null  int64  
dtypes: float64(3), int64(7), object(1)
memory usage: 872.6+ KB


In [5]:
df = df.iloc[:366,:]

In [22]:
data_encoder = df[['time_idx','year', 'month','day','hour', 'dayofweek_num']].astype(float)
data_decoder = df[['Energy_kwh','temperature']].astype(float)
data_output = df['Energy_kwh'].astype(float)

In [23]:
data_encoder = np.expand_dims(data_encoder, axis=0)
data_decoder = np.expand_dims(data_decoder, axis=0)
data_output = np.expand_dims(data_output, axis=0)
data_output = np.expand_dims(data_output, axis=-1)

In [24]:
class Encoder_layer(tf.keras.layers.Layer):
    def __init__(self, num_untis:int):
        super(Encoder_layer,self).__init__()


        self.LSTM_encoder = tf.keras.layers.LSTM(units=num_untis,
                                                 return_state = True,
                                                 name = 'LSTM_encoder')
        
    def call(self,inputs):
        
        output_encoder, state_h, state_c = self.LSTM_encoder(inputs)

        return output_encoder, state_h, state_c
    
class Decoder_layer(tf.keras.layers.Layer):
    def __init__(self, num_untis:int):
        super(Decoder_layer, self).__init__()

        self.LSTM_decoder = tf.keras.layers.LSTM(units=num_untis,
                                                 return_sequences=True,
                                                 return_state = True,
                                                 name = 'LSTM_decoder')

        self.Dense_output = tf.keras.layers.Dense(1)
    def call(self,inputs,encoder_states):


        output_decoder, state_h, state_c = self.LSTM_decoder(inputs,initial_state=encoder_states)
        output = self.Dense_output(output_decoder)

        return output
    
class Seq_to_Seq_LSTM(tf.keras.Model):
    def __init__(self, num_units:int):

        super().__init__()
        self.num_units = num_units


        self.encoder_layer = Encoder_layer(self.num_units)
        self.decoder_layer = Decoder_layer(self.num_units)


    def call(self,inputs):

        enc,dec= inputs
       
        _, state_h, state_c = self.encoder_layer(enc)

        output_decoder = self.decoder_layer(dec, encoder_states = [state_h, state_c])

        return output_decoder
    





In [25]:
model = Seq_to_Seq_LSTM(num_units=64)

optmin = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optmin, loss='mae')


model.fit([data_encoder,data_decoder],
          data_output,epochs=100,verbose=True,batch_size = 64)

Epoch 1/100


2024-06-03 08:02:55.400678: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 47.4837
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 47.3746
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 47.2817
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 47.1864
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 47.0861
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 47.0027
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 46.9198
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 46.8357
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 46.7506
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 46.6658
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 46.5791
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 46.4911
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 46.4017
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 46.3122
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 46.2245
Epoch 16/100
1/1 ━

In [27]:
df.tail()

,time_idx,Energy_kwh,house_hold,temperature,apparentTemperature,bool_weather_missing_values,year,month,day,hour,dayofweek_num
361,361,28.062,MAC000002,-1.02,-1.02,0,2013,1,16,1,2
362,362,24.647,MAC000002,-1.33,-3.67,0,2013,1,16,2,2
363,363,25.515,MAC000002,-1.77,-3.63,0,2013,1,16,3,2
364,364,26.360,MAC000002,-1.76,-3.83,0,2013,1,16,4,2
365,365,27.919,MAC000002,-2.02,-4.09,0,2013,1,16,5,2


In [35]:
data = df.to_numpy()

In [36]:
n_batches = 32

# Calculate the batch size (ensure it's an integer)
batch_size = data.shape[0] // n_batches

batches = []

# Loop through the data and create batches
for i in range(n_batches):
  start_idx = i * batch_size
  end_idx = start_idx + batch_size
  batch = data[start_idx:end_idx]
  batches.append(batch)

# Print the shapes of the first few batches (optional)
for i in range(3):
  print(f"Batch {i+1} shape:", batches[i].shape)

Batch 1 shape: (11, 11)
Batch 2 shape: (11, 11)
Batch 3 shape: (11, 11)
